In [10]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [13]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-02-19 21:06:38,049 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:06:38,742 INFO: Initializing external client
2025-02-19 21:06:38,742 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:06:40,005 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Fetching data from 2025-01-22 02:06:38.048575+00:00 to 2025-02-20 01:06:38.048575+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.87s) 


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-19 21:06:50,416 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:06:50,424 INFO: Initializing external client
2025-02-19 21:06:50,425 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:06:51,445 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,67,0.0,2025-02-20 03:00:00+00:00
1,147,0.0,2025-02-20 03:00:00+00:00
2,65,1.0,2025-02-20 03:00:00+00:00
3,156,0.0,2025-02-20 03:00:00+00:00
4,155,1.0,2025-02-20 03:00:00+00:00
...,...,...,...
248,207,0.0,2025-02-20 03:00:00+00:00
249,119,0.0,2025-02-20 03:00:00+00:00
250,12,1.0,2025-02-20 03:00:00+00:00
251,31,0.0,2025-02-20 03:00:00+00:00


In [17]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-02-19 21:06:53,267 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:06:53,274 INFO: Initializing external client
2025-02-19 21:06:53,275 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:06:54,262 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [18]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 253/253 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213633/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)